In [45]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import time
import itertools
import pickle
import json

from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

path_main_folder = '/home/antorosi/Documents/Prediction'
sys.path.append(path_main_folder)

from conso.load_shape_data import load_data_conso, get_uniformed_data_conso, change_granularity, get_x_y_prediction_conso, get_train_test_sets, normalized_dataset, select_variables, add_atypical_events_to_dict_data_conso
from models.feedforward_NN import FeedForward
from aed.atypical_event import AtypicalEventsList, AtypicalEvent, dataframe_daily_to_ael
from aed.detection import get_next_hd_events, sort_atypical_events, get_residuals, get_atypical_score, get_atypical_ds, prediction_conso_plot, aed_conso_plot
from aed.multi_period import get_prediction_results, get_aed_results, get_complete_df

### Load and shape data 

In [46]:
# Load
path_data = os.path.join(path_main_folder, 'data')
dict_data_conso = load_data_conso(path_data)

### Folder to store results  

In [47]:
# folder to store results
path_out = os.path.join(path_main_folder, 'out', 'iterate_FFNN_1')
if not os.path.exists(path_out):
    os.mkdir(path_out)

In [48]:
path_data = os.path.join("/local/home/antorosi/Documents/Prediction/data")
with open(os.path.join(path_data, 'ae_reference_list_wwe_2013-2017' + '.pickle'), 'rb') as f:
    ael_reference= pickle.load(f)

### Iterative fitting parameters 

In [49]:
# iterative parameters
nb_iter = 5
nb_ae = 50

# input parameters
selected_variables = ['calendar', 'conso','meteo', 'atypical_events']
gen_name = 'cmcaae'

# model parameters
nb_hidden_units = 171
nb_hidden_layers = 2
dropout = 0.1
training_epochs=400
batch_size=100

In [50]:
# atypical event list initialisation
ael = AtypicalEventsList()

In [51]:
# Test periods for each K step of the cross-validation
cv_periods = {}
cv_periods['period_1'] = (datetime.datetime(year=2013, month=1, day=1), datetime.datetime(year=2013, month=12, day=31))
cv_periods['period_2'] = (datetime.datetime(year=2014, month=1, day=1), datetime.datetime(year=2014, month=12, day=31))
cv_periods['period_3'] = (datetime.datetime(year=2015, month=1, day=1), datetime.datetime(year=2015, month=12, day=31))
cv_periods['period_4'] = (datetime.datetime(year=2016, month=1, day=1), datetime.datetime(year=2016, month=12, day=31))
cv_periods['period_5'] = (datetime.datetime(year=2017, month=1, day=1), datetime.datetime(year=2017, month=12, day=31))

### Main loop

In [44]:
for iteration in [3,4]:
    ### Create new folder to store iteration results
    path_out_iter = os.path.join(path_out, 'iter_'+str(iteration))
    if not os.path.exists(path_out_iter):
        os.mkdir(path_out_iter)
    
    ### UPDATE DATA WITH NEW ATYPICAL EVENTS ###
    dict_data_conso = add_atypical_events_to_dict_data_conso(dict_data_conso, ael)

    # Uniformization
    data_conso_df, dict_colnames_conso = get_uniformed_data_conso(dict_data_conso)

    # Granularity from 15 min to 1H
    data_conso_df = change_granularity(data_conso_df, granularity="1H")

    # Get x and y from prediction
    x_conso, y_conso, dict_colnames_conso = get_x_y_prediction_conso(data_conso_df, dict_colnames_conso, horizon_length=24)

    # Getting each datasets
    dict_datasets = {}
    for key, date_period in cv_periods.items():
        x_conso_selected_var = select_variables(x_conso, dict_colnames_conso, selected_variables)
        dataset, dict_ds = get_train_test_sets(x_conso_selected_var, y_conso, date_period[0], date_period[1])
        dataset = normalized_dataset(dataset, dict_colnames_conso)

        dict_datasets[key] = {'dataset': dataset, 'dict_ds': dict_ds}
    
    # Deleting useless info
    del x_conso, y_conso
    
    ### Getting FFNN input dim ###
    input_dim = dict_datasets['period_1']['dataset']['train']['x'].shape[1]

    ### Prepare results wrap up ###
    results_df = pd.DataFrame(columns=['name', 'layer_dims','dropout_rates','batchsize',
                                               'best_iter', 'train_mse',
                                               'train_mae', 'train_mape',
                                               'test_mse', 'test_mae',
                                               'test_mape'])
    path_results = path_out_iter
    
    ### TRAINNING ###
    
    idx = 0

    for name_period, el in dict_datasets.items():
        dataset = el['dataset']

        print('========================= Iteration {} Model {} ========================='.format(iteration+1,idx+1))

        # Prepare model characteristics
        name_model = '{}_FFNN_l{}*{}_d{}*{}_{}_norm'.format(name_period,nb_hidden_units, nb_hidden_layers,
                                                            dropout, nb_hidden_layers, 
                                                            gen_name)

        # Compile model
        model = FeedForward(name=name_model, output=path_out_iter, input_dim=input_dim, output_dim=1, 
                   l_dims=[nb_hidden_units]*nb_hidden_layers, dropout_rates=[dropout]*nb_hidden_layers,
                   loss = 'mean_squared_error', metrics = ['mape', 'mae'])

        # Prepare callbacks
        callbacks = []
        early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100,
                                                           verbose=0, mode='auto')

        model_checkpoint = ModelCheckpoint(os.path.join(path_out_iter, name_model, 'models', 'model-best.hdf5'),
                                                   monitor='val_loss',
                                                   verbose=0, save_best_only=True, save_weights_only=False,
                                                   mode='auto', period=1)

        tensorboard_model = TensorBoard(log_dir=os.path.join(path_out_iter, name_model, 'results', 'logs', time.strftime('%Y-%m-%d_%H:%M', time.localtime(time.time()))))
        tensorboard_summary = TensorBoard(log_dir=os.path.join(path_out_iter, 'logs', name_model))


        callbacks.append(early_stop)
        callbacks.append(model_checkpoint)
        callbacks.append(tensorboard_model)
        callbacks.append(tensorboard_summary)

        # Train model
        model.main_train(dataset, training_epochs=training_epochs, batch_size=batch_size, callbacks=callbacks)

        # Get result and put it in results
        _, result = model.analyze_history(dataset)

        results_df = results_df.append(result, ignore_index=True)
        results_df.to_csv(os.path.join(path_results, 'cv_results.csv'), sep=';')
        
        # Reset graph
        K.clear_session()
        import tensorflow as tf
        tf.reset_default_graph()

        idx += 1

    ### Atypical Event Detection ###
    path_models_folder = path_out_iter
    mode = 2
    type_model = 'keras'
    
    # get prediction
    prediction_results = get_prediction_results(path_models_folder, dict_datasets, mode, type_model)
    
    # get atypical events info
    threshold = 0.95
    aed_results, ael_full_model = get_aed_results(prediction_results, threshold)
    
    # get complete dataframe
    atypical_full_df, prediction_full_df = get_complete_df(prediction_results, aed_results)
    
    # sort events
    atypical_new_df = sort_atypical_events(atypical_full_df, ael_reference)
    
    # get ael
    selected_ae_iter = atypical_new_df[:nb_ae]
    ael_iter = dataframe_daily_to_ael(selected_ae_iter)
    
    # merge ael
    ael = ael.get_union(ael_iter)
    
    ### Save results ###

    # plot 
    # Not working with the current method
    path_plot = path_results
    name_plot = 'full_period_mode{}_t{}_iter{}.html'.format(mode, threshold, iteration)
    aed_conso_plot(data_conso_df, atypical_full_df, prediction_full_df, dict_colnames_conso, path_plot, name_plot)
    
    # data
    with open(os.path.join(path_results, 'atypical_full_df.json'),'wb') as f:
        pickle.dump(atypical_full_df,f)
    with open(os.path.join(path_results, 'prediction_full_df.json'),'wb') as f:
        pickle.dump(prediction_full_df,f)
    with open(os.path.join(path_results, 'ael.json'),'wb') as f:
        pickle.dump(ael,f)
    with open(os.path.join(path_results, 'ael_iter.json'),'wb') as f:
        pickle.dump(ael_iter,f)
    
    ael.get_events_list().to_csv(os.path.join(path_results, 'ael.csv'), sep=';')
    

========================= Iteration 4 Model 1 =========================
model: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_input (InputLayer)     (None, 156)               0         
_________________________________________________________________
dense_hidden_0 (Dense)       (None, 171)               26847     
_________________________________________________________________
dropout_hidden_0 (Dropout)   (None, 171)               0         
_________________________________________________________________
dense_hidden_1 (Dense)       (None, 171)               29412     
_________________________________________________________________
dropout_hidden_1 (Dropout)   (None, 171)               0         
_________________________________________________________________
dense_output (Dense)         (None, 1)                 172       
Total params: 56,431
Trainable params: 56,431
Non-trainable pa

Epoch 26/400
35108/35108 [==============================] - 1s 27us/step - loss: 9819571.8710 - mean_absolute_percentage_error: 4.6700 - mean_absolute_error: 2462.8298 - val_loss: 4624773.2773 - val_mean_absolute_percentage_error: 2.9763 - val_mean_absolute_error: 1579.2224
Epoch 27/400
35108/35108 [==============================] - 1s 27us/step - loss: 9830160.1028 - mean_absolute_percentage_error: 4.6855 - mean_absolute_error: 2468.4463 - val_loss: 4477451.4241 - val_mean_absolute_percentage_error: 2.9048 - val_mean_absolute_error: 1533.3546
Epoch 28/400
35108/35108 [==============================] - 1s 28us/step - loss: 9929155.3989 - mean_absolute_percentage_error: 4.6855 - mean_absolute_error: 2475.0229 - val_loss: 4257343.9810 - val_mean_absolute_percentage_error: 2.8037 - val_mean_absolute_error: 1479.7442
Epoch 29/400
35108/35108 [==============================] - 1s 26us/step - loss: 9780317.9153 - mean_absolute_percentage_error: 4.6515 - mean_absolute_error: 2453.6427 - val_l

Epoch 56/400
35108/35108 [==============================] - 1s 27us/step - loss: 8050524.3350 - mean_absolute_percentage_error: 4.1395 - mean_absolute_error: 2218.2218 - val_loss: 2542637.4887 - val_mean_absolute_percentage_error: 2.0302 - val_mean_absolute_error: 1106.8908
Epoch 57/400
35108/35108 [==============================] - 1s 28us/step - loss: 7923953.5812 - mean_absolute_percentage_error: 4.0989 - mean_absolute_error: 2198.6236 - val_loss: 2554075.2553 - val_mean_absolute_percentage_error: 2.0107 - val_mean_absolute_error: 1109.8453
Epoch 58/400
35108/35108 [==============================] - 1s 27us/step - loss: 7838303.7311 - mean_absolute_percentage_error: 4.0647 - mean_absolute_error: 2180.0663 - val_loss: 2645495.8604 - val_mean_absolute_percentage_error: 2.0321 - val_mean_absolute_error: 1119.2748
Epoch 59/400
35108/35108 [==============================] - 1s 27us/step - loss: 7898098.8342 - mean_absolute_percentage_error: 4.0772 - mean_absolute_error: 2187.7094 - val_l

Epoch 86/400
35108/35108 [==============================] - 1s 27us/step - loss: 7640123.0134 - mean_absolute_percentage_error: 4.0096 - mean_absolute_error: 2155.0519 - val_loss: 2154015.6384 - val_mean_absolute_percentage_error: 1.7428 - val_mean_absolute_error: 964.5016
Epoch 87/400
35108/35108 [==============================] - 1s 25us/step - loss: 7598326.2463 - mean_absolute_percentage_error: 4.0011 - mean_absolute_error: 2151.2790 - val_loss: 2667094.3268 - val_mean_absolute_percentage_error: 2.0769 - val_mean_absolute_error: 1155.1337
Epoch 88/400
35108/35108 [==============================] - 1s 29us/step - loss: 7667811.4312 - mean_absolute_percentage_error: 3.9928 - mean_absolute_error: 2152.5642 - val_loss: 2129404.4514 - val_mean_absolute_percentage_error: 1.7473 - val_mean_absolute_error: 957.1492
Epoch 89/400
35108/35108 [==============================] - 1s 27us/step - loss: 7592777.4976 - mean_absolute_percentage_error: 3.9807 - mean_absolute_error: 2140.7315 - val_los

Epoch 116/400
35108/35108 [==============================] - 1s 27us/step - loss: 7445250.2952 - mean_absolute_percentage_error: 3.9482 - mean_absolute_error: 2125.5290 - val_loss: 2333640.3675 - val_mean_absolute_percentage_error: 1.8809 - val_mean_absolute_error: 1038.0832
Epoch 117/400
35108/35108 [==============================] - 1s 27us/step - loss: 7538241.5910 - mean_absolute_percentage_error: 3.9473 - mean_absolute_error: 2128.1817 - val_loss: 2001983.7017 - val_mean_absolute_percentage_error: 1.6465 - val_mean_absolute_error: 915.6288
Epoch 118/400
35108/35108 [==============================] - 1s 28us/step - loss: 7411401.9005 - mean_absolute_percentage_error: 3.9524 - mean_absolute_error: 2124.9537 - val_loss: 2047194.4318 - val_mean_absolute_percentage_error: 1.6712 - val_mean_absolute_error: 925.2417
Epoch 119/400
35108/35108 [==============================] - 1s 29us/step - loss: 7413157.6140 - mean_absolute_percentage_error: 3.9383 - mean_absolute_error: 2120.6432 - val

...

Epoch 270/400
35132/35132 [==============================] - 1s 28us/step - loss: 6698003.8240 - mean_absolute_percentage_error: 3.6958 - mean_absolute_error: 2007.0309 - val_loss: 1091811.0252 - val_mean_absolute_percentage_error: 1.3408 - val_mean_absolute_error: 729.3567
Epoch 271/400
35132/35132 [==============================] - 1s 28us/step - loss: 6608461.8968 - mean_absolute_percentage_error: 3.6720 - mean_absolute_error: 1993.8013 - val_loss: 1083827.6442 - val_mean_absolute_percentage_error: 1.3172 - val_mean_absolute_error: 717.2529
Epoch 272/400
35132/35132 [==============================] - 1s 28us/step - loss: 6728652.5920 - mean_absolute_percentage_error: 3.7130 - mean_absolute_error: 2015.9300 - val_loss: 1186527.8638 - val_mean_absolute_percentage_error: 1.4501 - val_mean_absolute_error: 777.7844
Epoch 273/400
35132/35132 [==============================] - 1s 28us/step - loss: 6608080.6997 - mean_absolute_percentage_error: 3.6838 - mean_absolute_error: 1997.3549 - val_

Epoch 300/400
35132/35132 [==============================] - 1s 28us/step - loss: 6659281.8326 - mean_absolute_percentage_error: 3.7002 - mean_absolute_error: 2007.8724 - val_loss: 1183086.2763 - val_mean_absolute_percentage_error: 1.3878 - val_mean_absolute_error: 759.7277
Epoch 301/400
35132/35132 [==============================] - 1s 28us/step - loss: 6601151.6518 - mean_absolute_percentage_error: 3.6871 - mean_absolute_error: 1998.9257 - val_loss: 1145676.2041 - val_mean_absolute_percentage_error: 1.3280 - val_mean_absolute_error: 732.5917
Epoch 302/400
35132/35132 [==============================] - 1s 27us/step - loss: 6614888.6605 - mean_absolute_percentage_error: 3.6686 - mean_absolute_error: 1990.6611 - val_loss: 1044138.3069 - val_mean_absolute_percentage_error: 1.3014 - val_mean_absolute_error: 705.1811
Epoch 303/400
35132/35132 [==============================] - 1s 30us/step - loss: 6712336.8905 - mean_absolute_percentage_error: 3.7067 - mean_absolute_error: 2010.6371 - val_

Epoch 330/400
35132/35132 [==============================] - 1s 28us/step - loss: 6627155.5747 - mean_absolute_percentage_error: 3.6770 - mean_absolute_error: 1997.5781 - val_loss: 1325972.8212 - val_mean_absolute_percentage_error: 1.6185 - val_mean_absolute_error: 853.9470
Epoch 331/400
35132/35132 [==============================] - 1s 27us/step - loss: 6590731.0265 - mean_absolute_percentage_error: 3.6825 - mean_absolute_error: 1997.8662 - val_loss: 1047115.9501 - val_mean_absolute_percentage_error: 1.3243 - val_mean_absolute_error: 717.8376
Epoch 332/400
35132/35132 [==============================] - 1s 28us/step - loss: 6495504.7842 - mean_absolute_percentage_error: 3.6540 - mean_absolute_error: 1983.9135 - val_loss: 1213254.4842 - val_mean_absolute_percentage_error: 1.4699 - val_mean_absolute_error: 788.2968
Epoch 333/400
35132/35132 [==============================] - 1s 27us/step - loss: 6558250.5896 - mean_absolute_percentage_error: 3.6825 - mean_absolute_error: 1998.3891 - val_

Epoch 360/400
35132/35132 [==============================] - 1s 27us/step - loss: 6541052.0878 - mean_absolute_percentage_error: 3.6699 - mean_absolute_error: 1990.9530 - val_loss: 1246531.8019 - val_mean_absolute_percentage_error: 1.4956 - val_mean_absolute_error: 806.2864
Epoch 361/400
35132/35132 [==============================] - 1s 28us/step - loss: 6520130.9970 - mean_absolute_percentage_error: 3.6541 - mean_absolute_error: 1984.6416 - val_loss: 1021686.0564 - val_mean_absolute_percentage_error: 1.3036 - val_mean_absolute_error: 702.6434
Epoch 362/400
35132/35132 [==============================] - 1s 28us/step - loss: 6599856.3739 - mean_absolute_percentage_error: 3.6691 - mean_absolute_error: 1994.9914 - val_loss: 1220860.8882 - val_mean_absolute_percentage_error: 1.4371 - val_mean_absolute_error: 787.3044
Epoch 363/400
35132/35132 [==============================] - 1s 27us/step - loss: 6563530.3159 - mean_absolute_percentage_error: 3.6687 - mean_absolute_error: 1994.0691 - val_

Epoch 390/400
35132/35132 [==============================] - 1s 27us/step - loss: 6373305.7921 - mean_absolute_percentage_error: 3.6242 - mean_absolute_error: 1965.8964 - val_loss: 1142864.0221 - val_mean_absolute_percentage_error: 1.3447 - val_mean_absolute_error: 740.9707
Epoch 391/400
35132/35132 [==============================] - 1s 27us/step - loss: 6456283.3533 - mean_absolute_percentage_error: 3.6494 - mean_absolute_error: 1979.5627 - val_loss: 1396650.5720 - val_mean_absolute_percentage_error: 1.5231 - val_mean_absolute_error: 846.2656
Epoch 392/400
35132/35132 [==============================] - 1s 28us/step - loss: 6512741.5983 - mean_absolute_percentage_error: 3.6531 - mean_absolute_error: 1983.8938 - val_loss: 1011317.6137 - val_mean_absolute_percentage_error: 1.2771 - val_mean_absolute_error: 691.1418
Epoch 393/400
35132/35132 [==============================] - 1s 27us/step - loss: 6512988.5709 - mean_absolute_percentage_error: 3.6581 - mean_absolute_error: 1985.9075 - val_